In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

In [39]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_taet = x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [40]:
x_train[0].shape

(28, 28, 1)

In [41]:
type(x_train[0][0,0,0])

numpy.float64

In [42]:
train_daatset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [43]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10)


    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x

model = MyModel()

In [44]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [45]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [46]:
@tf.function
def train(images, labels):
    with tf.GradientTape() as tape:
        pred = model(images)
        loss = loss_function(labels, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, pred)

In [47]:
@tf.function
def test(images, labels):
    pred = model(images)
    loss = loss_function(labels, pred)
    test_loss(loss)
    test_accuracy(labels, pred)

In [ ]:
epochs = 5

for epoch in range(epochs):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    #tarin
    for images, labels in train_daatset:
        train(images, labels)

    #test
    for images, labels in test_dataset:
        test(images, labels)

    print("epoch:", epoch + 1,
          f"Train Loss:{train_loss.result()}",
          f"Train Acc:{train_accuracy.result()}",
          f"test Loss:{test_loss.result()}",
          f"test Acc:{test_accuracy.result()}")

In [ ]:
model.save("my_model")

In [ ]:
model = tf.keras.models.load_model = "my_model"